# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [22]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [28]:
!cat talks.tsv

type	date	title	venue	location	talk_url	venue_url	description
Plenary/Keynote Talk	2021-07-20	Mixed Integer Bilevel Optimization	SIAM Conference on Optimization (OP21)		https://meetings.siam.org/sess/dsp_programsess.cfm?SESSIONCODE=72554	https://www.siam.org/conferences/cm/program/invited-presentations/op21-invited-presentations	
Plenary/Keynote Talk	2021-02-25	Panel Discussion Real-World Optimisation Models: Successes and Pitfalls	Beale Lecture, The OR Society		https://youtu.be/jBQCOSKL1ho?t=1780	https://www.theorsociety.com/events/beale-lecture/	
Plenary/Keynote Talk	2021-02-12	New integer and Bilevel Formulations for the k-Vertex Cut Problem	5th AIRO Young Workshop		https://www.youtube.com/watch?v=LXjE8PWxaPQ&feature=youtu.be	http://opslab.dieti.unina.it/index.php/en/laboratory?id=59	
Plenary/Keynote Talk	2020-05-18	Casting Light on the Hidden Bilevel Structure of the Capacitated Vertex Separator Problem	Mixed Integer Programming Workshop 2020 Online Edition (MIP2020)		https://www.y

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [29]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,type,date,title,venue,location,talk_url,venue_url,description
0,Plenary/Keynote Talk,2021-07-20,Mixed Integer Bilevel Optimization,SIAM Conference on Optimization (OP21),NaN,https://meetings.siam.org/sess/dsp_programsess...,https://www.siam.org/conferences/cm/program/in...,NaN
1,Plenary/Keynote Talk,2021-02-25,Panel Discussion Real-World Optimisation Model...,"Beale Lecture, The OR Society",NaN,https://youtu.be/jBQCOSKL1ho?t=1780,https://www.theorsociety.com/events/beale-lect...,NaN
2,Plenary/Keynote Talk,2021-02-12,New integer and Bilevel Formulations for the k...,5th AIRO Young Workshop,NaN,https://www.youtube.com/watch?v=LXjE8PWxaPQ&fe...,http://opslab.dieti.unina.it/index.php/en/labo...,NaN
3,Plenary/Keynote Talk,2020-05-18,Casting Light on the Hidden Bilevel Structure ...,Mixed Integer Programming Workshop 2020 Online...,NaN,https://www.youtube.com/watch?v=dl9KW0CnQ74,https://sites.google.com/view/mipworkshop2020/...,NaN
4,Plenary/Keynote Talk,2019-06-23,From Game Theory to Graph Theory: A Bilevel Jo...,EURO 2019: 30th European Conference on Operati...,"UCD, Dublin, Ireland",slides/EURO2019_Ljubic.pdf,https://www.euro2019dublin.com/tutorial-speakers,NaN
5,Plenary/Keynote Talk,2019-01-30,Solving Very Large Scale Covering Location Pro...,IX International Workshop on Locational Analys...,"Cádiz, Spain",slides/IWOLOCA2019_Ljubic.pdf,http://redloca.ulpgc.es/redloca18.html,NaN
6,Plenary/Keynote Talk,2018-09-12,From Game Theory to Graph Theory: A Bilevel Jo...,OR 2018: International Conference on Operation...,"Brussels, Belgium",https://www.or2018.be/slides/Ljubic.pdf,https://www.or2018.be/welcome/index,NaN
7,Plenary/Keynote Talk,2018-06-18,Branch-and-Cut Algorithms for Mixed-Integer Bi...,2nd International Workshop on Bilevel Programming,"Inria Lille-Nord Europe, Lille, France",NaN,https://iwobip2.sciencesconf.org/,NaN
8,Plenary/Keynote Talk,2018-06-18,Recent Developments on Exact Solvers for the (...,22nd Belgian Mathematical Programming Workshop,"La-Roche-en-Ardennes, Belgium",slides/COMEX_Ljubic_Full.pdf,http://comex.ulb.ac.be/?page_id=103,NaN
9,Talk,2020-08-30,New integer and Bilevel Formulations for the k...,22ème séminaire du groupe POC (online),NaN,https://www.youtube.com/watch?v=v66IrAejl7k&fe...,https://www.lamsade.dauphine.fr/poc/?q=node/72,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [30]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [31]:
loc_dict = {}

for row, item in talks.iterrows():
    url_slug = '-'.join([i for i in item.title.lower().split()])
    
    md_filename = str(item.date) + "-" + url_slug + ".md"
    html_filename = str(item.date) + "-" + url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    if len(str(item.venue)) > 3:
        if len(str(item.venue_url)) > 3:
            md += "\nThis talk was part of [" + item.venue + "](" + item.venue_url + ")" 
        else:
            md += "\nThis talk was part of " + item.venue

        if len(str(item.talk_url)) > 3:
            if 'slides' in str(item.talk_url):
                md += " and more information can be found [here]({{site.url}}/docs/" + item.talk_url + ")\n" 
            else:
                md += " and more information can be found [here](" + item.talk_url + ")\n" 
        else:
            md += "."
    else:
        if len(str(item.talk_url)) > 3:
            md += "\nMore information can be found [here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [32]:
!ls ../_talks

2006-07-31-optimization-in-telecommunication-networks:-multicommodity-flow,-facility-location-and-steiner-tree-problems.md
2007-03-19-connected-facility-location-in-the-design-of-telecommunication-networks.md
2010-06-27-two-stage-branch-&-cut-for-two-stage-stochastic-network-design-problems.md
2010-08-16-dissaggregated-flow-formulation-for-the-single-source-network-loading-problem.md
2010-08-16-opttelnets:-algorithmische-ansätze.md
2011-10-21-the-generalized-regenerator-location-problem.md
2012-01-27-solving-two-stage-stochastic-steiner-tree-problems-by-two-stage-branch-and-cut.md
2012-04-04-optimization-tools-for-last-mile-access-networks.md
2012-04-26-the-recoverable-robust-two-level-network-design-problem.md
2012-07-04-solving-2-stage-stochastic-network-design-problems-by-2-stage-branch-and-cut.md
2012-11-22-the-maximum-weight-connected-subgraph-problem.md
2013-08-29-the-maximum-weight-connected-subgraph-problem:-applications-in-bioinformatics.md
2014-05-13-the-recoverable-robust-fa

In [15]:
!cat ../_talks/2012-03-01-mixed-integer-bilevel-optimization.md

---
title: "Mixed Integer Bilevel Optimization"
collection: talks
type: "Talk"
permalink: /talks/2012-03-01-mixed-integer-bilevel-optimization
venue: "SIAM Conference on Optimization (OP21)"
---
This talk was part of [SIAM Conference on Optimization (OP21)](https://www.siam.org/conferences/cm/program/invited-presentations/op21-invited-presentations) and more information can be found [here](https://meetings.siam.org/sess/dsp_programsess.cfm?SESSIONCODE=72554)
